In [4]:
import pandas as pd

import requests
from requests import get
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
import time
import timeit


In [5]:
import re
headers = {"Accept-Language": "en-US, en;q=0.5"}
direction = ["North","East","West","South","E","W","N","S"]

In [8]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver=webdriver.Chrome(executable_path = 'C:/Users/kuria/anaconda3/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe',options = options)
driver.get("https://www.rentcollegepads.com/off-campus-housing/ohio-state/search")
listings=[]

while(True):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        stages = driver.find_elements_by_class_name('col-9')
        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        container = soup.find_all('div', attrs={'class':'col-9'})
    
        len_list_before = len(listings)
  
        for div in container:
            if div.a["href"] in listings:
                continue
            else:
                listings.append(div.a['href'])

        len_list_after = len(listings)
    
        if len_list_before==len_list_after:
            break

        loadmore = driver.find_element_by_css_selector('button[class = "btn btn-success btn-block"]')
        loadmore.click()
        time.sleep(2)
        
    except NoSuchElementException: 
        print("Reached the end")
        end = timeit.timeit()
        break
        

Reached the end
0.0005301999999574036


In [2]:

df = pd.read_csv('Listing_1.csv')

In [181]:
name = []
location_name = []
location = []
stuff = []
names = []
bed = []
bath = []
rent = []
available = []
sq_ft = []

for i in range(0,len(df),1):
    url = df['urls'][i]
    page = requests.get(url ,headers = headers)
    soup = BeautifulSoup(page.text, "html.parser")    
    table = soup.find_all('div', 
              attrs = {'class':'tab-pane active',
                      'aria-hidden':"false",
                      'id':"__BVID__43"})

    for row in table[0].find_all('tr'):
        for col in row.find_all('td'):
            if not col.string is None:
                stuff.append(col.string.strip())
            else:
                stuff.append("None")
         
        if not stuff:
            continue
        else:
            names.append(stuff[0])
            bed.append(stuff[1])
            bath.append(stuff[2])
            rent.append(stuff[3])
            sq_ft.append(stuff[4])
            available.append(stuff[5])
        
        location_name.append(soup.find("span", re.compile("sub-heading")).text)
        name = soup.find("span", re.compile("sub-heading")).text
        if not list(set(name.split()) & set(direction)):
            location.append("Null")
        else:
            location.append(list(set(name.split()) & set(direction))[0])
    
        stuff = []


In [182]:
units = pd.DataFrame(columns = ['location_name','location','name','beds','baths','price','available','sq_ft'])

units['location_name'] = location_name
units['location'] = location
units['name'] = names
units['beds'] = bed
units['baths'] = bath
units['price'] = rent
units['available'] = available
units['sq_ft'] = sq_ft

In [183]:
units

,location_name,location,name,beds,baths,price,available,sq_ft
0,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Unfurnished,2,1,790-790,2020-08-15,None
1,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Furnished,2,1,850-850,2020-08-15,None
2,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Duo,2,2,1130-1140,0000-00-00,960
3,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 1,4,4,950-965,0000-00-00,1549
4,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 2,4,4,920-955,0000-00-00,1481
...,...,...,...,...,...,...,...,...
1389,"\n155 W 10th Ave, Columbus, OH 43201, United S...",W,None,6,2,0,0000-00-00,None
1390,"\n393 W 8th Ave Columbus, OH 43201\n",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None
1391,"\n50 W Oakland Ave, Columbus, OH 43201, United...",W,4,1,1,599-599,2020-07-04,None
1392,"\n201 E 16th Ave Columbus, OH 43201 USA\n",E,5x2,5,2,2500-2500,2018-12-01,1352


In [217]:
stuff = []
parking_type = []
number = []
price = []
price_type = []
location_name = []
random_0 = []
random_1 = []
random_2 = []
random_3 = []

for i in range(0,len(df),1):
    url = df['urls'][i]
    page = requests.get(url ,headers = headers)
    soup = BeautifulSoup(page.text, "html.parser")
    feature_block = soup.find_all('div',
             attrs = {'class':'feature-block'})[1]
    
    check_description = feature_block.find_all('h2')
    if not check_description[0].string :
        for row in feature_block.find_all('tr'):
            for col in row.find_all('td'):
                if not col.string is None:
                    stuff.append(col.string.strip())
                else:
                    stuff.append("None")
         
            if not stuff:
                continue
            else:
                parking_type.append(stuff[0])
                number.append(stuff[1])
                price.append(stuff[2])
                price_type.append(stuff[3])

            stuff = []
            
            location_name.append(soup.find("span", re.compile("sub-heading")).text)
            
    else:

        for li in feature_block.find_all('li'):
            if 'Laundry' in li.text:
                random_1.append(li.text)
                random_1.append(url)
                random_1.append(soup.find("span", re.compile("sub-heading")).text)
            else:
                random_1.append('Null')
            if 'Furnished' in li.text:
                random_2.append(li.text)
                random_2.append(url)
                random_2.append(soup.find("span", re.compile("sub-heading")).text)
            else:
                random_2.append('Null')
            if 'Parking' in li.text:
                random_3.append(li.text)
                random_3.append(url)
                random_3.append(soup.find("span", re.compile("sub-heading")).text)
            else:
                random_3.append('Null')
        
       # random.append(soup.find("span", re.compile("sub-heading")).text)
    

In [219]:
parking = pd.DataFrame(columns = ['location_name','parking_type', 'number','price','price_type'])
parking['location_name'] = location_name
parking['parking_type'] = parking_type
parking['number'] = number
parking['price_parking'] = price
parking['price_type'] = price_type

In [222]:
units

,location_name,location,name,beds,baths,price,available,sq_ft
0,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Unfurnished,2,1,790-790,2020-08-15,None
1,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Furnished,2,1,850-850,2020-08-15,None
2,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Duo,2,2,1130-1140,0000-00-00,960
3,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 1,4,4,950-965,0000-00-00,1549
4,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 2,4,4,920-955,0000-00-00,1481
...,...,...,...,...,...,...,...,...
1389,"\n155 W 10th Ave, Columbus, OH 43201, United S...",W,None,6,2,0,0000-00-00,None
1390,"\n393 W 8th Ave Columbus, OH 43201\n",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None
1391,"\n50 W Oakland Ave, Columbus, OH 43201, United...",W,4,1,1,599-599,2020-07-04,None
1392,"\n201 E 16th Ave Columbus, OH 43201 USA\n",E,5x2,5,2,2500-2500,2018-12-01,1352


In [223]:
parking

,location_name,parking_type,number,price,price_type
0,"\n442 East Northwood Avenue Columbus, OH 43201...",Off-Street Parking,1,$100,Included
1,"\n250 W Norwich Ave Columbus, OH 43201\n",Garage Parking,1,$100,Monthly
2,"\n1234 Steelwood Rd Columbus, OH 43212\n",Surface Lot,1,Call For Price,FREE
3,"\n1234 Steelwood Rd Columbus, OH 43212\n",Garage Parking,1,$50,Monthly
4,"\n1234 Steelwood Rd Columbus, OH 43212\n",Indoor Bike Storage,2,Call For Price,FREE
...,...,...,...,...,...
499,"\n1656 Summit St, Columbus, OH 43201, United S...",Off-Street Parking,0,Call For Price,Included
500,"\n43 East 15th Avenue Columbus, OH 43201 USA\n",Off-Street Parking,0,Call For Price,Included
501,"\n50 W Oakland Ave, Columbus, OH 43201, United...",Off-Street Parking,0,Call For Price,Included
502,"\n201 E 16th Ave Columbus, OH 43201 USA\n",Off-Street Parking,2,$300,Annually


In [319]:
parking_new = pd.DataFrame(columns = ['location_name','parking_type','number'])
loc_name = []
park = []
i = 0
while  i < len(random_3): 
    if random_3[i] != 'Null':
        loc_name.append(random_3[i+2])
        park.append(random_3[i])
        i+=3
    else:
        i+=1

parking_new['location_name'] = loc_name
parking_new['parking_type'] = park
parking_new.loc[:,'number'] = 1
parking_new

,location_name,parking_type,number
0,"\n151 W 10th Ave, Columbus, OH 43201, United S...",On Street Parking Only,1
1,"\n116 E 12th Ave, Columbus, OH 43201, United S...",Off Street Parking,1
2,"\n253 E 19th Ave, Columbus, OH 43201, United S...",Assigned Parking Space,1
3,"\n23 W 10th Ave\nColumbus, OH 43201\n",Off Street Parking,1
4,"\n95 Chittenden Ave, Columbus, OH 43201, Unite...",2 Assigned Parking Spaces (1 Double Lane),1
5,"\n94 E 18th Ave, Columbus, OH 43201, United St...",2 Assigned Parking Spaces (1 Double Lane),1
6,"\n214 W Norwich Ave, Columbus, OH 43201, Unite...",Assigned Off-Street Parking Spaces,1
7,"\n155 W 10th Ave, Columbus, OH 43201, United S...",On Street Parking Only,1


In [321]:
parking = pd.concat([parking, parking_new], axis=0)

In [333]:
parking

,location_name,parking_type,number,parking_price,price_type
0,"\n442 East Northwood Avenue Columbus, OH 43201...",Off-Street Parking,1,$100,Included
1,"\n250 W Norwich Ave Columbus, OH 43201\n",Garage Parking,1,$100,Monthly
2,"\n1234 Steelwood Rd Columbus, OH 43212\n",Surface Lot,1,Call For Price,FREE
3,"\n1234 Steelwood Rd Columbus, OH 43212\n",Garage Parking,1,$50,Monthly
4,"\n1234 Steelwood Rd Columbus, OH 43212\n",Indoor Bike Storage,2,Call For Price,FREE
...,...,...,...,...,...
3,"\n23 W 10th Ave\nColumbus, OH 43201\n",Off Street Parking,1,NaN,NaN
4,"\n95 Chittenden Ave, Columbus, OH 43201, Unite...",2 Assigned Parking Spaces (1 Double Lane),1,NaN,NaN
5,"\n94 E 18th Ave, Columbus, OH 43201, United St...",2 Assigned Parking Spaces (1 Double Lane),1,NaN,NaN
6,"\n214 W Norwich Ave, Columbus, OH 43201, Unite...",Assigned Off-Street Parking Spaces,1,NaN,NaN


In [328]:
units

,location_name,location,name,beds,baths,price,available,sq_ft
0,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Unfurnished,2,1,790-790,2020-08-15,None
1,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Furnished,2,1,850-850,2020-08-15,None
2,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Duo,2,2,1130-1140,0000-00-00,960
3,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 1,4,4,950-965,0000-00-00,1549
4,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 2,4,4,920-955,0000-00-00,1481
...,...,...,...,...,...,...,...,...
1389,"\n155 W 10th Ave, Columbus, OH 43201, United S...",W,None,6,2,0,0000-00-00,None
1390,"\n393 W 8th Ave Columbus, OH 43201\n",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None
1391,"\n50 W Oakland Ave, Columbus, OH 43201, United...",W,4,1,1,599-599,2020-07-04,None
1392,"\n201 E 16th Ave Columbus, OH 43201 USA\n",E,5x2,5,2,2500-2500,2018-12-01,1352


In [334]:
final_list = pd.merge(units, parking, on='location_name', how='outer')

In [335]:
pd.merge(units, parking, on='location_name', how='outer')

,location_name,location,name,beds,baths,price,available,sq_ft,parking_type,number,parking_price,price_type
0,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Unfurnished,2,1,790-790,2020-08-15,None,Off-Street Parking,1,$100,Included
1,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Furnished,2,1,850-850,2020-08-15,None,Off-Street Parking,1,$100,Included
2,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Duo,2,2,1130-1140,0000-00-00,960,Garage Parking,1,$100,Monthly
3,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 1,4,4,950-965,0000-00-00,1549,Garage Parking,1,$100,Monthly
4,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 2,4,4,920-955,0000-00-00,1481,Garage Parking,1,$100,Monthly
...,...,...,...,...,...,...,...,...,...,...,...,...
1477,"\n155 W 10th Ave, Columbus, OH 43201, United S...",W,None,6,2,0,0000-00-00,None,On Street Parking Only,1,NaN,NaN
1478,"\n393 W 8th Ave Columbus, OH 43201\n",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None,NaN,NaN,NaN,NaN
1479,"\n50 W Oakland Ave, Columbus, OH 43201, United...",W,4,1,1,599-599,2020-07-04,None,Off-Street Parking,0,Call For Price,Included
1480,"\n201 E 16th Ave Columbus, OH 43201 USA\n",E,5x2,5,2,2500-2500,2018-12-01,1352,Off-Street Parking,2,$300,Annually


In [337]:
final_list.to_csv('final_list.csv')

In [365]:
laundry = pd.DataFrame(columns = ['location_name','laundry_aval'])
loc_name = []
laundry_aval = []
i = 0
while  i < len(random_1): 
    if random_1[i] != 'Null':
        loc_name.append(random_1[i+2])
        laundry_aval.append(random_1[i])
        i+=3
    else:
        i+=1
laundry['location_name'] = loc_name
laundry['laundry_aval'] = laundry_aval
laundry

,location_name,laundry_aval
0,"\n51 E 13th Ave, Columbus, OH 43201, USA\n",Laundry in building
1,"\n151 W 10th Ave, Columbus, OH 43201, United S...",Laundry in Apartment
2,"\n116 E 12th Ave, Columbus, OH 43201, United S...",Laundry in House
3,"\n141 E 13th Ave, Columbus, OH 43201, USA\n",Laundry in Building
4,"\n253 E 19th Ave, Columbus, OH 43201, United S...",Laundry in Unit
5,"\n95 Chittenden Ave, Columbus, OH 43201, Unite...",Laundry in Unit
6,"\n94 E 18th Ave, Columbus, OH 43201, United St...",Laundry in Apartment
7,"\n99 E 16th Ave, Columbus, OH 43201, USA\n",Laundry on site
8,"\n214 W Norwich Ave, Columbus, OH 43201, Unite...",Laundry in Building
9,"\n155 W 10th Ave, Columbus, OH 43201, United S...",Laundry in Apartment


In [369]:
final_list = pd.merge(final_list,laundry,on = "location_name",how = "outer")

In [370]:
final_list.to_csv('final_list.csv')

In [359]:
parking['parking_type'] = np.where((df['location'] == df['']) & (df['one'] <= df['three'])
                     , df['one'], np.nan)

['\n151 W 10th Ave, Columbus, OH 43201, United States\n',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null']

In [358]:
laundry_aval

['Laundry in Apartment',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 '\n99 E 16th Ave, Columbus, OH 43201, USA\n',
 'Null']

In [354]:
random_3

['Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'On Street Parking Only',
 'https://www.rentcollegepads.com/city/columbus-oh/listings/151-w-10th-ave-47660',
 '\n151 W 10th Ave, Columbus, OH 43201, United States\n',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Off Street Parking',
 'https://www.rentcollegepads.com/city/columbus-oh/listings/116-e-12th-ave-47669',
 '\n116 E 12th Ave, Columbus, OH 43201, United States\n',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Assigned Parking Space',
 'https://www.rentcollegepads.com/city/columbus-oh/listings/253-e-19th-ave-47624',
 '\n253 E 19th Ave, Columbus, OH 43201, United States\n',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Nul

In [372]:
import pickle
final_list.to_pickle("Please_work") 

In [373]:
df = pd.read_pickle("Please_work")

In [374]:
df

,location_name,location,name,beds,baths,price,available,sq_ft,parking_type,number,parking_price,price_type,laundry_aval
0,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Unfurnished,2,1,790-790,2020-08-15,None,Off-Street Parking,1,$100,Included,NaN
1,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Furnished,2,1,850-850,2020-08-15,None,Off-Street Parking,1,$100,Included,NaN
2,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Duo,2,2,1130-1140,0000-00-00,960,Garage Parking,1,$100,Monthly,NaN
3,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 1,4,4,950-965,0000-00-00,1549,Garage Parking,1,$100,Monthly,NaN
4,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 2,4,4,920-955,0000-00-00,1481,Garage Parking,1,$100,Monthly,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,"\n155 W 10th Ave, Columbus, OH 43201, United S...",W,None,6,2,0,0000-00-00,None,On Street Parking Only,1,NaN,NaN,Laundry in Apartment
1478,"\n393 W 8th Ave Columbus, OH 43201\n",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None,NaN,NaN,NaN,NaN,NaN
1479,"\n50 W Oakland Ave, Columbus, OH 43201, United...",W,4,1,1,599-599,2020-07-04,None,Off-Street Parking,0,Call For Price,Included,NaN
1480,"\n201 E 16th Ave Columbus, OH 43201 USA\n",E,5x2,5,2,2500-2500,2018-12-01,1352,Off-Street Parking,2,$300,Annually,NaN
